In [4]:
%matplotlib inline

In [47]:
import numpy as np
import pandas as pd
import datetime as dt
import matplotlib.pyplot as plt
import time
import fnmatch
import os
from shutil import copyfile

## Read in coordinates of the powerplant locations

In [6]:
Location = pd.ExcelFile('US_Power_Plant_Locations.xlsx')
Location1 = Location.parse('Sheet2')

## Find corresponding grid cells

In [ ]:
def round_to_05(x):
    correction = 0.5
    np.round(np.array(Location1[' Facility Latitude'] / (1/8)))

In [32]:
prepare_lat = np.round( np.array(Location1[' Facility Latitude'] / (1/8)) ) - np.array(Location1[' Facility Latitude'] / (1/8))
prepare_lat[prepare_lat > 0] =0.5
prepare_lat[prepare_lat <= 0] = -0.5

prepare_lon = np.round( np.array(Location1[' Facility Longitude'] / (1/8)) ) - np.array(Location1[' Facility Longitude'] / (1/8))
prepare_lon[prepare_lon > 0] =0.5
prepare_lon[prepare_lon <= 0] = -0.5

grid_lat = (np.round( np.array(Location1[' Facility Latitude'] / (1/8)) ) - prepare_lat)/8 
grid_lon = (np.round( np.array(Location1[' Facility Longitude'] / (1/8)) ) - prepare_lon)/8 

In [33]:
Location1['Grid Lat'] = grid_lat
Location1['Gird Lon'] = grid_lon

### Verify that we do find corresponding grid cells correctly

In [41]:
all(Location1[' Facility Latitude']- grid_lat < 0.0625) ,all(Location1[' Facility Longitude'] - grid_lon < 0.0625)

(True, True)

## Copy chosen meteorological parameter files into speicific directory

In [62]:
list_area = os.listdir('./VIC_Forcing_Maurer/')

In [66]:
List = []
for i in range(len(grid_lat)):
#for i in [1]:
    data_name = 'data_' + str(grid_lat[i]) + '_' + str(grid_lon[i])
    for j in range(len(list_area)):
        file_path = './VIC_Forcing_Maurer/'+list_area[j] + '/'
        for file in os.listdir(file_path):
            if fnmatch.fnmatch(file, data_name):
                copyfile(file_path + data_name, './forcing_US_powerplant/'+ data_name)
                print(i)
                List.append(i)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
76
77
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
224
225
226
227
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
254
255
256
257
258
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277
278
279
28

### First, number of powerplants, which found meteo parameter files, is less than that Paulina gave us because our data is in continental USA, some of their powerplants is located in Hawaii and Alaska, or Pacific Ocean.

### Second, we find that the number of meteo parameter files in the forcing directory is less than the number of powerplants in continental USA. After checking the location files, we find some powerplants located in the same grid cell. 

In [107]:
Location_chosen = Location1.loc[List]

mylist = []

for i in range(Location_chosen.shape[0]):
    no = List[i]
    mylist.append((grid_lat[no],grid_lon[no]))

len(set(mylist))

1149

## Create soil parameter file

In [137]:
Soil_raw = pd.read_csv('./param/make_soil_param/all.soil.wsne.with_NR',sep = ' ',header = None, index_col= [0])

In [150]:
set_list = list(set(mylist))

chosen_DF = pd.DataFrame()
for i in range(len(set_list)):
    lat = set_list[i][0]
    lon = set_list[i][1]
    temp_DF = Soil_raw.loc[(Soil_raw[3] == lat) & (Soil_raw[4] ==  lon)]
    chosen_DF = pd.concat([chosen_DF,temp_DF])

In [158]:
chosen_DF = chosen_DF.sort_index()

In [163]:
chosen_DF.to_csv('./param/soil.wsne.US_powerplant',sep = ' ', header=None, index=None ,encoding = 'utf-8')

,1,2,3,4,5,6,7,8,9,10,...,45,46,47,48,49,50,51,52,53,54
0,,,,,,,,,,,,,,,,,,,,,
515,1,81213,46.9375,-123.4375,0.3,0.10,10,0.75,2,10.585,...,0.200,0.200,0.010,0.03,2283.140,0,0,0,0,16.6538
665,1,78431,46.1875,-123.1875,0.3,0.10,10,0.75,2,10.585,...,0.200,0.200,0.010,0.03,1678.390,0,0,0,0,16.5507
832,1,78433,46.1875,-122.9375,0.3,0.10,10,0.75,2,10.585,...,0.200,0.200,0.010,0.03,1352.900,0,0,0,0,17.1577
835,1,79825,46.5625,-122.9375,0.3,0.10,10,0.75,2,17.954,...,0.360,0.360,0.010,0.03,1360.600,0,0,0,0,17.5222
921,1,80754,46.8125,-122.8125,0.3,0.10,10,0.75,2,10.585,...,0.200,0.200,0.010,0.03,1291.970,0,0,0,0,17.3732
998,1,76579,45.6875,-122.6875,0.3,0.10,10,0.75,2,10.585,...,0.200,0.200,0.010,0.03,1098.370,0,0,0,0,18.2923
1019,1,88179,48.8125,-122.6875,0.3,0.10,10,0.75,2,10.585,...,0.200,0.200,0.010,0.03,1012.060,0,0,0,0,16.5353
1141,1,57557,40.5625,-122.4375,0.1,0.01,10,0.75,2,13.603,...,0.320,0.320,0.010,0.03,886.445,0,0,0,0,27.7958
1202,1,86789,48.4375,-122.4375,0.3,0.10,10,0.75,2,10.585,...,0.200,0.200,0.010,0.03,839.486,0,0,0,0,16.3365
